In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
train_df = pd.read_csv("datasets/train.csv").set_index("Id")
train_df.drpna(axis=1, subset=["Alley", "PoolQC"])
corr_matrix = train_df.corr()
X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"].copy()
# num_X = X.select_dtypes(include="number")
# cat_X = X.select_dtypes(include="category")

In [63]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [65]:
train_df["FireplaceQu"].value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64